In [15]:
import os
import json

In [16]:
def get_afId(af):
    if isinstance(af, list):
        return
    elif isinstance(af, dict):
        return {af['@id']}

In [17]:
def get_refCount(data):
    if data['abstracts-retrieval-response']['item']['bibrecord']['tail'] is None:
        return None
    else:
        d = data['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['@refcount']
        if isinstance(d, str):
            # print(d)
            return d
        else:
            return None

In [18]:
def get_refPublishYearTitleText(data):
    if data['abstracts-retrieval-response']['item']['bibrecord']['tail'] is None:
        return None
    else:
        d = data['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference']
        # print(d)
        if isinstance(d, list):
            # print(d)
            p = {}
            for ref in d:
                # print(ref)
                # print(ref['ref-info'])
                if 'ref-publicationyear' in ref['ref-info'].keys():
                    if ref['ref-info']['ref-publicationyear']['@first'] in p.keys():
                        pass
                    else:
                        p[ref['ref-info']['ref-publicationyear']['@first']] = []
                else:
                    if 'Unknown' in p.keys():
                        pass
                    else:
                        p['Unknown'] = []

                if 'ref-title'in ref['ref-info'].keys() and 'ref-publicationyear'in ref['ref-info'].keys():
                    p[ref['ref-info']['ref-publicationyear']['@first']].append(ref['ref-info']['ref-title']['ref-titletext'])
                elif 'ref-publicationyear'in ref['ref-info'].keys():
                    # p[ref['ref-info']['ref-publicationyear']['@first']].append('Unknown')
                    pass
                elif 'ref-title'in ref['ref-info'].keys():
                    p['Unknown'].append(ref['ref-info']['ref-title']['ref-titletext'])
                else:
                    # p['Unknown'].append('Unknown')
                    pass
            return p
        elif isinstance(d, dict):
            p = {}
            ref = d
            if 'ref-publicationyear' in ref['ref-info'].keys():
                if ref['ref-info']['ref-publicationyear']['@first'] in p.keys():
                    pass
                else:
                    p[ref['ref-info']['ref-publicationyear']['@first']] = []
            else:
                if 'Unknown' in p.keys():
                    pass
                else:
                    p['Unknown'] = []

            if 'ref-title'in ref['ref-info'].keys() and 'ref-publicationyear'in ref['ref-info'].keys():
                p[ref['ref-info']['ref-publicationyear']['@first']].append(ref['ref-info']['ref-title']['ref-titletext'])
            elif 'ref-publicationyear'in ref['ref-info'].keys():
                # p[ref['ref-info']['ref-publicationyear']['@first']].append('Unknown')
                pass
            elif 'ref-title'in ref['ref-info'].keys():
                p['Unknown'].append(ref['ref-info']['ref-title']['ref-titletext'])
            else:
                # p['Unknown'].append('Unknown')
                pass
            return p
        else:
            return None

In [19]:
def get_abstracts(data):
    if data['abstracts-retrieval-response']['item']['bibrecord']['head'] is None:
        return None
    else:
        d = data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
        if isinstance(d, str):
            # print(d)
            return d
        else:
            return None

In [20]:
def get_person(data):
    if data['abstracts-retrieval-response']['item']['bibrecord']['head'] is None:
        return None
    elif 'correspondence' not in data['abstracts-retrieval-response']['item']['bibrecord']['head'].keys():
        return None
    else:
        d = data['abstracts-retrieval-response']['item']['bibrecord']['head']['correspondence']
        people = []
        if isinstance(d, list):
            # print(d)
            for cor in d:
                people.append(cor['person']['ce:indexed-name'])
            return people
        elif isinstance(d, dict) :
            people.append(d['person']['ce:indexed-name'])
            return people

In [21]:
def get_correspondence(data):
    if data['abstracts-retrieval-response']['item']['bibrecord']['head'] is None:
        return None
    elif 'correspondence' not in data['abstracts-retrieval-response']['item']['bibrecord']['head'].keys():
        return None
    else:
        d = data['abstracts-retrieval-response']['item']['bibrecord']['head']['correspondence']
        c = {}
        if isinstance(d, list):
            # print(d)
            for cor in d:
                if 'person' not in cor.keys():
                    if 'Unknown' in c.keys():
                        c['Unknown'].append(cor['affiliation']['ce:source-text'])
                    elif 'Unknown' not in c.keys():
                        c['Unknown'] = []
                        c['Unknown'].append(cor['affiliation']['ce:source-text'])
                elif 'affiliation' not in cor.keys():
                    # c[cor['person']['ce:indexed-name']] = 'Unknown'
                    c[cor['person']['ce:indexed-name']] = None
                else:
                    if 'ce:source-text' in cor['affiliation'].keys():
                        c[cor['person']['ce:indexed-name']] = cor['affiliation']['ce:source-text']
                    elif 'ce:source-text' not in cor['affiliation'].keys():
                        # c[cor['person']['ce:indexed-name']] = 'Unknown'
                        c[cor['person']['ce:indexed-name']] = None
            return c
        elif isinstance(d, dict) :
            cor = d
            if 'person' not in cor.keys():
                if 'Unknown' in c.keys():
                    c['Unknown'].append(cor['affiliation']['ce:source-text'])
                elif 'Unknown' not in c.keys():
                    c['Unknown'] = []
                    c['Unknown'].append(cor['affiliation']['ce:source-text'])
            elif 'affiliation' not in cor.keys():
                # c[cor['person']['ce:indexed-name']] = 'Unknown'
                c[cor['person']['ce:indexed-name']] = None
            else:
                if 'ce:source-text' in cor['affiliation'].keys():
                    c[cor['person']['ce:indexed-name']] = cor['affiliation']['ce:source-text']
                elif 'ce:source-text' not in cor['affiliation'].keys():
                    # c[cor['person']['ce:indexed-name']] = 'Unknown'
                    c[cor['person']['ce:indexed-name']] = None
            return c

In [22]:
def get_affiliation(data):
    a = {}
    aff = data['abstracts-retrieval-response']['affiliation']
    if isinstance(aff,list):
        for af in aff:
            a[af['@id']] = {
                'name': af['affilname'],
                'city': af['affiliation-city'],
                'country': af['affiliation-country']
            }
    elif isinstance(aff, dict):
        af = aff
        a[af['@id']] = {
            'name': af['affilname'],
            'city': af['affiliation-city'],
            'country': af['affiliation-country']
        }
    return a

In [43]:
def get_publishedDate(data):
    date = data['abstracts-retrieval-response']['item']['bibrecord']['head']['source']['publicationdate']
    if ('year' in date.keys() and 'month' in date.keys() and 'day' in date.keys()):
        y = date['year']
        m = date['month']
        d = date['day']
    else:
        return None
    return y + '-' + m + '-' + d

In [24]:
# import json
# with open('201800000.json') as f:
#     d = json.load(f)
# get_publishedDate(d)

In [25]:
def get_title(data):
    if 'dc:title' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['dc:title']

In [26]:
def get_aggreationType(data):
    if 'prism:aggregationType' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['prism:aggregationType']

In [27]:
def get_subtypeDescription(data):
    if 'subtypeDescription' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['subtypeDescription']

In [28]:
def get_endingPage(data):
    if 'prism:endingPage' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['prism:endingPage']

In [29]:
def get_sourceID(data):
    if 'source-id' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['source-id']

In [30]:
def get_citedbyCount(data):
    if 'citedby-count' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['citedby-count']

In [31]:
def get_publisher(data):
    if 'dc:publisher' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['dc:publisher']

In [32]:
def get_publisherCopyright(data):
    if 'publishercopyright' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['publishercopyright']

In [33]:
def get_issueIdentifier(data):
    if 'prism:issueIdentifier' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['prism:issueIdentifier']

In [34]:
def get_identifier(data):
    if 'dc:identifier' not in data['abstracts-retrieval-response']['coredata'].keys():
        return None
    else:
        return data['abstracts-retrieval-response']['coredata']['dc:identifier']

In [35]:
def get_language(data):
    if 'language' not in data['abstracts-retrieval-response'].keys():
        return None
    else:
        if data['abstracts-retrieval-response']['language'] is None:
            return None
        else:
            return data['abstracts-retrieval-response']['language']['@xml:lang']

In [36]:
def get_authorKeywords(data):
    if 'authkeywords' not in data['abstracts-retrieval-response'].keys():
        return None
    elif data['abstracts-retrieval-response']['authkeywords'] is None:
        return None
    else:
        kw = data['abstracts-retrieval-response']['authkeywords']['author-keyword']
        key = []
        if isinstance(kw, list):
            for k in kw:
                key.append(k['$'])
        elif isinstance(kw, dict):
            k = kw
            key.append(k['$'])
        return key

In [37]:
def get_subjectArea(data):
    if 'subject-areas' not in data['abstracts-retrieval-response'].keys():
        return None
    elif data['abstracts-retrieval-response']['subject-areas']['subject-area'] is None:
        return None
    else:
        sa = data['abstracts-retrieval-response']['subject-areas']['subject-area']
        area = {}
        if isinstance(sa, list):
            for s in sa:
                if s['@abbrev'] not in area.keys():
                    area[s['@abbrev']] = []
                    area[s['@abbrev']].append(s['$'])
                elif s['@abbrev'] in area.keys():
                    area[s['@abbrev']].append(s['$'])
        elif isinstance(sa, dict):
            s = sa
            if s['@abbrev'] not in area.keys():
                area[s['@abbrev']] = []
                area[s['@abbrev']].append(s['$'])
            elif s['@abbrev'] in area.keys():
                area[s['@abbrev']].append(s['$'])
        return area
        

In [38]:
def get_author(data):
    if 'authors' not in data['abstracts-retrieval-response'].keys():
        return None
    elif data['abstracts-retrieval-response']['authors']['author'] is None:
        return None
    else:
        au = data['abstracts-retrieval-response']['authors']['author']
        author = {}
        if isinstance(au, list):
            for a in au:
                if 'affiliation' in a.keys():
                    if isinstance(a['affiliation'],list):
                        author[a['@auid']] = {
                            'name': a['preferred-name']['ce:indexed-name'],
                            'afid': a['affiliation'][-1]['@id']
                        }
                    elif isinstance(a['affiliation'], dict):
                        author[a['@auid']] = {
                            'name': a['preferred-name']['ce:indexed-name'],
                            'afid': a['affiliation']['@id']
                        }
                elif 'affiliation' not in a.keys():
                    author[a['@auid']] = {
                            'name': a['preferred-name']['ce:indexed-name'],
                            'afid': None
                        }
        if isinstance(au, dict):
            a = au
            if 'affiliation' in a.keys():
                if isinstance(a['affiliation'],list):
                    author[a['@auid']] = {
                        'name': a['preferred-name']['ce:indexed-name'],
                        'afid': a['affiliation'][-1]['@id']
                    }
                elif isinstance(a['affiliation'], dict):
                    author[a['@auid']] = {
                        'name': a['preferred-name']['ce:indexed-name'],
                        'afid': a['affiliation']['@id']
                    }
            elif 'affiliation' not in a.keys():
                author[a['@auid']] = {
                        'name': a['preferred-name']['ce:indexed-name'],
                        'afid': None
                    }
        return author

In [44]:
i = 1
for year in os.listdir(r'Data'):
    if not year.startswith('.'):
        for file in os.listdir(f'Data/{year}'):
            print(file)
            with open(os.path.join('Data', year, file), 'r') as f:
                # file = f.read()
                # print(json.load(f))
                data = json.load(f)
                ref_count = get_refCount(data)
                ref_publishYear_titleText = get_refPublishYearTitleText(data)
                abstracts = get_abstracts(data)
                correspondence = get_correspondence(data)
                affiliation = get_affiliation(data)
                publishedDate = get_publishedDate(data)
                title = get_title(data)
                aggregationType = get_aggreationType(data)
                subtypeDescription = get_subtypeDescription(data)
                endingPage = get_endingPage(data)
                sourceID = get_sourceID(data)
                citedByCount = get_citedbyCount(data)
                publisher = get_publisher(data)
                publisherCopyright = get_publisherCopyright(data)
                issueIdentifier = get_issueIdentifier(data)
                identifier = get_identifier(data)
                language = get_language(data)
                authorKeywords = get_authorKeywords(data)
                subjectArea = get_subjectArea(data)
                author = get_author(data)

                document = {
                    'reference' : {
                        'ref_count': ref_count, 
                        'ref_publishYear_titleText': ref_publishYear_titleText, 
                    },
                    'abstracts' : abstracts,
                    'correspondence' : correspondence,
                    'affiliation' : affiliation,
                    'publishedDate' : publishedDate,
                    'coreData' : {
                        'title': title, 
                        'aggregationType': aggregationType, 
                        'subtypeDescription': subtypeDescription,
                        'endingPage': endingPage, 
                        'sourceID': sourceID, 
                        'citedByCount': citedByCount,
                        'publisher': publisher, 
                        'publisherCopyright': publisherCopyright, 
                        'issueIdentifier': issueIdentifier, 
                        'identifier': identifier,
                        },
                    'language' : language,
                    'authorKeywords' : authorKeywords,
                    'subjectArea' : subjectArea,
                    'author' : author,
                }
                
                with open(os.path.join('SelectedData', str(i)+'.json'), 'w') as ff:
                    json.dump(document, ff, indent=4)
                ff.close()
                i += 1
                # break

202203039
202203805
202201904
202201138
202202749
202200822
202200648
202202923
202203462
202201335
202201107
202202782
202203650
202204011
202201563
202203234
202203006
202204223
202201751
202200683
202201597
202200445
202202312
202202120
202200677
202202544
202203496
202200213
202200021
202202776
202202118
202202924
202200825
202200019
202202588
202203668
202201903
202204029
202200489
202201769
202203802
202200026
202202771
202202543
202203491
202200214
202202127
202200670
202201590
202200442
202202315
202203001
202204224
202201756
202200684
202204016
202201564
202203233
202201100
202202785
202203657
202203465
202201332
202203498
202200813
202202778
202201599
202202912
202200679
202203008
202203834
202201935
202201109
202200646
202201794
202202111
202202323
202200474
202203695
202202747
202200010
202200222
202202575
202203661
202201136
202201304
202203453
202202581
202204212
202201760
202203037
202203205
202204020
202200480
202201552
202201932
202203659
202203833
202201758
202204018
